# ERK-KTR Full FOV Stimulation Pipeline

## System Init

### Load pymmcore and required python libraries

In [1]:
import os
import time

os.environ["QT_LOGGING_RULES"] = (
    "*.debug=false; *.warning=false"  # Fix to suppress PyQT warnings from napari-micromanager when running in a Jupyter notebook
)
os.environ["MICROMANAGER_PATH"] = "C:\\Program Files\\Micro-Manager-2.0"

from fov import FOV
from useq import MDAEvent
import pandas as pd
import numpy as np
import random
import napari
import pymmcore_plus
from napari_micromanager import MainWindow

from utils import create_folders
from useq._mda_event import SLMImage

mmc = pymmcore_plus.CMMCorePlus()

### Device Specific Init for Niesen Microscope

In [2]:
mmc.loadSystemConfiguration(
    "C:\\Program Files\Micro-Manager-2.0\\MMConfig_demo.cfg"
)
mmc.setConfig(groupName="System", configName="Startup")
mmc.setChannelGroup(channelGroup="Channel")

## GUI - Napari Micromanager

### Load GUI

In [3]:
### Base GUI ###
viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)

In [4]:
### Add MDA widget for FOV selection ###
from pymmcore_widgets.mda import MDAWidget

mdawidget = MDAWidget(mmcore=mmc)
viewer.window.add_dock_widget(mdawidget)

### Functions to break and re-connect link with GUI if manually broken

The following functions can be used to manually interrupt to connection between the GUI and the running rtm-pymmcore script. However, normally you don't need to execute them. 

In [ ]:
### Break connection
# mm_wdg._core_link.cleanup()

In [ ]:
### Manually reconnect pymmcore with napari-micromanager
from napari_micromanager._core_link import CoreViewerLink

mm_wdg._core_link = CoreViewerLink(viewer, mmc)

## Create a DF with all planned acquisitions and stimulations

### Settings for Experiment

In [15]:
df_acquire = pd.DataFrame(
    columns=[
        "fov",
        "timestep",
        "time",
        "time_experiment",
        "treatment",
        "acquired",
        "stim",
        "channels",
        "channel_stim",
    ]
)

base_path = (
    "C:\\Users\\Alex\\Ausbildung\\PhD_temp\\test_exp"
)
experiment_name = "exp_test"
path = os.path.join(base_path, experiment_name)

N_FRAMES = 2
FIRST_FRAME_STIMULATION = 10

create_folders(
    path, ["stim", "raw", "labels", "stim_mask", "tracks", "labels_rings", "particles"]
)

time_between_frames = 2  # time in seconds between frames
time_per_fov = 1  # time in seconds per fov

timesteps = range(N_FRAMES)
channels = ["DAPI", "Cy5"]  # channel for segmentation first
channels_exposure = [150, 100]

# cell_lines = ["optoFGFR_high"] * 24 + ["optoFGFR"] * 24
cell_lines = ["FGFR_high"]
# if defining individual fovs, else these values a re ignored:
n_fovs_per_cell_line = 36  ## change this variable to the amount of fovs that you have per cell line. If only one cell line is set, this value will
# automatically set to total amount of fovs. If you are working will wellplate, this value will be ignored, as each columns
# will be an entry in the cell lines list.

n_fovs_per_well = 6  ## change this variable to the amount of fovs that you have per well. Set to None if you are not working with wellplate.
# If you are working with the autogenetreated wellplate, this value will be ignored, as each columns will be an entry in the cell lines list.

stim_exposures = [60]  # list of possible exposures in ms
# stim_timesteps = [list(range(10,N_FRAMES,1)), list(range(10,N_FRAMES,2)), list(range(10,N_FRAMES,10)), list(range(10,N_FRAMES,20)), [10], list(range(10,N_FRAMES,5))]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]
stim_timesteps = []
# stim_timesteps = [list(range(FIRST_FRAME_STIMULATION,N_FRAMES,2))]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]

# stim_timesteps = [[10], list(range(10, 60, 2)), list(range(10, 60, 1)), list(range(10,60,10)), list(range(10, 60, 20)), list(range(10, 60, 5))]  # list of timesteps for stimulation, if e.g. double stimulation in frame 0 and 1 is needed write [[0,1]]
stim_profiles = [
    {
        "channel": "FITC",
    }
]
stim_treatment = [
    {
        "stim_property": "global",
        "stim_profile": stim_profile,
        "stim_exposure": stim_exposure,
        "stim_timestep": stim_timestep,
    }
    for stim_profile in stim_profiles
    for stim_exposure in stim_exposures
    for stim_timestep in stim_timesteps
]

data_mda_fovs = None
stim_treatment

Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\stim already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\raw already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\labels already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\stim_mask already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\tracks already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\labels_rings already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\particles already exists


[]

In [16]:
channels_exposure = [150, 150]
channels_power = [80, 20]

### Map Experiment to FOVs

#### If FOVs already saved - Reload them from file

In [17]:
import json

file = os.path.join(path, "fovs.json")
with open(file, "r") as f:
    data_mda_fovs = json.load(f)

Only select one of the following two code blocks. If you autogenerated FOVs using the wellplate option of the MDA widget, then use the first code block, else the second. 

#### FOVs were manually selected using MDA widget

In [18]:
fovs: list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value().stage_positions
n_fovs = len(data_mda_fovs)
n_stim_treatments = len(stim_treatment)

if n_stim_treatments == 0: 
    stim_treatment_tot =n_fovs * [{"stim_timestep": [], "stim_exposure": None, "stim_profile": None}]
else: 
    n_fovs_per_stim_condition = n_fovs // n_stim_treatments // len(np.unique(cell_lines))


    stim_treatment_tot = []
    random.shuffle(stim_treatment)
    if n_fovs_per_well is not None:
        for stim_treat in stim_treatment:
            stim_treatment_tot.extend([stim_treat] * n_fovs_per_well)

    else:
        for i in range(0, n_fovs_per_stim_condition):
            stim_treatment_tot.extend(stim_treatment)
        random.shuffle(stim_treatment_tot)

        if n_fovs % n_stim_treatments != 0:
            print(
                f"Warning: Not equal number of fovs per stim condition. {n_fovs % n_stim_treatments} fovs will have repeated treatment"
            )
            stim_treatment_tot.extend(stim_treatment[: n_fovs % n_stim_treatments])
    print(f"Doing {n_fovs_per_stim_condition} replicates per stim condition")

if len(cell_lines) == 1:
    n_fovs_per_cell_line = n_fovs
else:
    stim_treatment_tot = stim_treatment_tot * len(np.unique(cell_lines))


for i, row in enumerate(data_mda_fovs):
    row = dict(row)
    if len(cell_lines) == 1:
        cell_line = cell_lines[0]
    else:
        cell_line = cell_lines[i // n_fovs_per_cell_line]
    fov = FOV(
        pos=(row["x"], row["y"]),
        index=i,
        name=str(i),
        path=path,
        metadata={"cell_line": cell_line},
        treatment=stim_treatment_tot[i],
    )
    fovs.append(fov)



### Use FOVs to generate dataframe for acquisition

In [19]:
n_fovs_simultaneously = time_between_frames // time_per_fov
start_time = 0

dfs = []
for fov in fovs:
    fov_group = fov.index // n_fovs_simultaneously
    start_time = fov_group * time_between_frames * len(timesteps)

    for timestep in timesteps:
        new_row = {
            "fov_object": fov,
            "fov": fov.index,
            "name": fov.name,
            "timestep": timestep,
            "time": start_time + timestep * time_between_frames,
            "treatment": fov.treatment,
            "metadata": fov.metadata,
            "stim": timestep in fov.treatment["stim_timestep"]
            and fov.treatment["stim_exposure"]
            != 0,  # not really important, only for visualisation
            "channels": channels,
            "channels_exposure": channels_exposure,
            "channel_power": channels_power,
            "stim_exposure": fov.treatment[
                "stim_exposure"
            ],  # not really important, only for visualisation
            "fname": f"{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}",
        }
        dfs.append(new_row)

df_acquire = pd.DataFrame(dfs)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", True)
df_acquire = df_acquire.sort_values(by=["time", "fov"])
print(f"Total Experiment Time: {df_acquire['time'].max()/3600}h")
df_acquire

Total Experiment Time: 0.0005555555555555556h


,fov_object,fov,name,timestep,time,treatment,metadata,stim,channels,channels_exposure,channel_power,stim_exposure,fname
0,<fov.FOV object at 0x00000216F3A20250>,0,0,0,0,"{'stim_timestep': [], 'stim_exposure': None, '...",{'cell_line': 'FGFR_high'},False,"[DAPI, Cy5]","[150, 150]","[80, 20]",None,000_00000
2,<fov.FOV object at 0x00000216F3A231D0>,1,1,0,0,"{'stim_timestep': [], 'stim_exposure': None, '...",{'cell_line': 'FGFR_high'},False,"[DAPI, Cy5]","[150, 150]","[80, 20]",None,001_00000
1,<fov.FOV object at 0x00000216F3A20250>,0,0,1,2,"{'stim_timestep': [], 'stim_exposure': None, '...",{'cell_line': 'FGFR_high'},False,"[DAPI, Cy5]","[150, 150]","[80, 20]",None,000_00001
3,<fov.FOV object at 0x00000216F3A231D0>,1,1,1,2,"{'stim_timestep': [], 'stim_exposure': None, '...",{'cell_line': 'FGFR_high'},False,"[DAPI, Cy5]","[150, 150]","[80, 20]",None,001_00001


## Run experiment

In [21]:
%load_ext autoreload
%autoreload 2

SLEEP_TIME_IN_HOURS = 0

for _ in range(0, SLEEP_TIME_IN_HOURS * 3600):
    time.sleep(1)
from add_frame import ImageProcessingPipeline
from segmentation.base_segmentator import DummySegmentator
from stimulation.base_stim import StimWholeFOV
from controller import ControllerSimulated, Analyzer
from tracking.base_tracker import TrackerNoTracking
from dmd import DMD
from queue import Queue

try:
    mm_wdg._core_link.cleanup()
except:
    pass

segmentator = DummySegmentator()
stimulator = StimWholeFOV()
tracker = TrackerNoTracking()

pipeline = ImageProcessingPipeline(segmentator, stimulator, tracker)
analyzer = Analyzer(pipeline)
queue = Queue()
controller = ControllerSimulated(analyzer, mmc, queue, project_patah="C:\\Users\\Alex\\Ausbildung\\PhD_temp\\test_exp\\old_data")
controller.run(df_acquire)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[02.04.2025 09:08:03] INFO     MDA Started: GeneratorMDASequence()                                   ]8;id=740385;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=257068;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#376\376]8;;\

                      INFO     index={'t': 0, 'c': 0, 'p': 0} channel=Channel(config='DAPI')         ]8;id=878585;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=669322;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\
                               exposure=150.0 min_start_time=0.0 x_pos=0.0 y_pos=0.0                               
                               metadata={'fov_object': <fov.FOV object at 0x00000216F3A20250>,                     
                               'fov': 0, 'name': '0', 'timestep': 0, 'time': 0, 'treatment':                       
                               {'stim_timestep': [], 'stim_exposure': None, 'stim_profile': None},                 
                               'metadata': {'cell_line': 'FGFR_high'}, 'stim': False, 'channels':                  
                               ['DAPI', 'Cy5'], 'channels_exposure': [150, 150], 'channel_power':                  
                               [80, 20], 'stim_exposure': None, 'fname': '000_00000', 'img_type':                  
                               <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'DAPI'}                     

                      INFO     index={'t': 0, 'c': 1, 'p': 0} channel=Channel(config='Cy5')          ]8;id=898143;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=423477;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\
                               exposure=150.0 min_start_time=0.0 x_pos=0.0 y_pos=0.0                               
                               metadata={'fov_object': <fov.FOV object at 0x00000216F3A20250>,                     
                               'fov': 0, 'name': '0', 'timestep': 0, 'time': 0, 'treatment':                       
                               {'stim_timestep': [], 'stim_exposure': None, 'stim_profile': None},                 
                               'metadata': {'cell_line': 'FGFR_high'}, 'stim': False, 'channels':                  
                               ['DAPI', 'Cy5'], 'channels_exposure': [150, 150], 'channel_power':                  
                               [80, 20], 'stim_exposure': None, 'fname': '000_00000', 'img_type':                  
                               <ImgType.IMG_RAW: 1>, 'last_channel': True, 'channel': 'Cy5'}                       

                      INFO     index={'t': 0, 'c': 0, 'p': 1} channel=Channel(config='DAPI')         ]8;id=103569;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=380046;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#337\337]8;;\
                               exposure=150.0 min_start_time=0.0 x_pos=20.01 y_pos=0.0                             
                               metadata={'fov_object': <fov.FOV object at 0x00000216F3A231D0>,                     
                               'fov': 1, 'name': '1', 'timestep': 0, 'time': 0, 'treatment':                       
                               {'stim_timestep': [], 'stim_exposure': None, 'stim_profile': None},                 
                               'metadata': {'cell_line': 'FGFR_high'}, 'stim': False, 'channels':                  
                               ['DAPI', 'Cy5'], 'channels_exposure': [150, 150], 'channel_power':                  
                               [80, 20], 'stim_exposure': None, 'fname': '001_00000', 'img_type':                  
                               <ImgType.IMG_RAW: 1>, 'last_channel': False, 'channel': 'DAPI'}                     

...

                      INFO     MDA Finished: GeneratorMDASequence()                                  ]8;id=699892;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=647041;file://c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pymmcore_plus\mda\_runner.py#463\463]8;;\

Exception in thread Thread-16 (run):
Traceback (most recent call last):
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Alex\Programmierung\01_git\PhD\rtm-pymmcore-1\add_frame.py", line 131, in run
    df_tracked = df_tracked.astype(df_datatypes)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pandas\core\generic.py", line 6605, in astype
    raise KeyError(
KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'timestep' not found in columns."


Empty DataFrame
Columns: [mean_intensity_C0_nuc, mean_intensity_C1_nuc, label, x, y, mean_intensity_C0_ring, mean_intensity_C1_ring, particle, frame, stim, time]
Index: []


Exception in thread Thread-17 (run):
Traceback (most recent call last):
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Alex\Programmierung\01_git\PhD\rtm-pymmcore-1\add_frame.py", line 131, in run
    df_tracked = df_tracked.astype(df_datatypes)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Alex\miniforge3\envs\pymmcore\Lib\site-packages\pandas\core\generic.py", line 6605, in astype
    raise KeyError(
KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'timestep' not found in columns."


Empty DataFrame
Columns: [mean_intensity_C0_nuc, mean_intensity_C1_nuc, label, x, y, mean_intensity_C0_ring, mean_intensity_C1_ring, particle, frame, stim, time]
Index: []
